In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-may-2022/train.csv
/kaggle/input/tabular-playground-series-may-2022/test.csv


In [4]:
# Load date sets.

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
test_ids = test_data['id'] 


train_data.head()


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


In [5]:
#check for unique values
for col in test_data:
    print(col)
    print(test_data[col].unique())

id
[ 900000  900001  900002 ... 1599997 1599998 1599999]
f_00
[ 0.44251689 -0.60559756  0.30398973 ... -0.33170369 -2.03107297
 -0.08590636]
f_01
[ 1.74380235e-01 -3.05714509e-01  2.44511024e+00 ... -3.28844814e-01
 -1.23839802e+00 -2.12377493e-03]
f_02
[-0.99981593  0.62766725  0.24651454 ... -1.18550262  0.96469876
  2.22737467]
f_03
[ 0.76274101 -0.57889756  0.81824834 ...  1.02212795 -1.04595013
  0.21714491]
f_04
[ 0.18677843 -1.75093096  0.35973054 ... -0.48309949  0.90606413
  3.17915307]
f_05
[-1.0747749   1.35555045 -1.33184544 ... -0.10714629  0.63430066
 -1.66018827]
f_06
[ 0.50188752 -0.19091089  1.35862229 ... -0.96828118 -0.70747429
  0.89198907]
f_07
[ 6  1  3  0  2  5  8  4  7 12  9 10 11 13 15 16]
f_08
[ 6  3  0  2  1  4  5  8 11  7 10  9 12 13]
f_09
[ 0  4  2  3  9  5  1  6  7  8 10 12 11 13 16 14]
f_10
[ 0  1  2  4  3  6  5  7  8  9 10 12 11 14 15 13]
f_11
[ 0  2  1  3  5  4  7  6  8  9 11 10 12 13 14]
f_12
[ 5  1  3  4  6  2  7  0  9  8 10 11 12 13 14 15]
f_13
[ 1  

In [6]:
#look for any NAN values
train_data.isna().sum()

id        0
f_00      0
f_01      0
f_02      0
f_03      0
f_04      0
f_05      0
f_06      0
f_07      0
f_08      0
f_09      0
f_10      0
f_11      0
f_12      0
f_13      0
f_14      0
f_15      0
f_16      0
f_17      0
f_18      0
f_19      0
f_20      0
f_21      0
f_22      0
f_23      0
f_24      0
f_25      0
f_26      0
f_27      0
f_28      0
f_29      0
f_30      0
target    0
dtype: int64